### This is a seq to seq model using encoders and decoders.Encoder and Decoder embedding matrices will be created which will be used for lookup of word representation for encoder inputs as well as decoder outputs. Here out input will be a seq of numbers and output will be the same sequence shifted by 1 . Output of the encoder will be used as input to decoder.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import time
import os

In [2]:
tf.__version__

'1.2.1'

In [3]:
BATCH_SIZE = 50
num_units = 512
MAX_SEQLEN = 10
encoder_vocabulary_size = 23
encoder_embedding_size = 10
decoder_vocabulary_size = 23
decoder_embedding_size = 10
NLAYERS = 3


### Encoder Cell Specifications

In [4]:
## Encoder inputs first get converted into a word representation
cells = [tf.contrib.rnn.GRUCell(num_units) for _ in range(NLAYERS)]
encoder_cell = tf.contrib.rnn.MultiRNNCell(cells)
ENCODER_INPUTS = tf.placeholder(dtype=tf.int32,shape=[None,None],name="Encoder_inputs") #BATCH_SIZE X SEQLEN
##embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
encoder_embedings = tf.get_variable(name="ENCODER_EMBEDDINGS",shape=[encoder_vocabulary_size,encoder_embedding_size],initializer=tf.contrib.layers.xavier_initializer(uniform=True,seed=None,dtype=tf.float32),
                                     dtype=tf.float32)
embed = tf.nn.embedding_lookup(encoder_embedings,ENCODER_INPUTS)

In [5]:
EncoderSeqLength= tf.placeholder(dtype=tf.int32,shape=[None])
output,state = tf.nn.dynamic_rnn(encoder_cell,embed,dtype=tf.float32,sequence_length=EncoderSeqLength)


### Decoder Cell Specifications

In [6]:

DecoderSeqLength = tf.placeholder(dtype=tf.int32,shape=[None]);
DECODER_INPUTS = tf.placeholder(dtype=tf.int32,shape=[None,None],name="DecoderInputs")

decoder_embeddings = tf.get_variable(name="DECODER_EMBEDDINGS",shape=[decoder_vocabulary_size,decoder_embedding_size],initializer=tf.contrib.layers.xavier_initializer(uniform=True,seed=None,dtype=tf.float32),
                                     dtype=tf.float32)
decoder_embed = tf.nn.embedding_lookup(decoder_embeddings,DECODER_INPUTS)
##A helper for use during training. Only reads inputs of dimension decoder_embedding_size 
##Returned sample_ids are the argmax of the RNN output logits.
helper = tf.contrib.seq2seq.TrainingHelper(decoder_embed,sequence_length=DecoderSeqLength)
decoder_output_layer = tf.contrib.keras.layers.Dense(decoder_vocabulary_size)
## implementation of dense layer : output = activation(dot(input, kernel) + bias) where activation is the element-wise 
##activation function passed as the activation argument, kernel is a weights matrix created by the layer, 
##and bias is a bias vector created by the layer (only applicable if use_bias is True).
##input paramater will be num_units that is size of output logit 
## If you don't specify any activation function, no activation is applied(ie. "linear" activation: `a(x) = x`)
## So here the no activation is applied. Later we will apply the softmax layer on this


### Attention Mechanism and do dynamic decoding

In [7]:
####Apply Attention
##creates a class providing functionality of storing the memory(Encoder output states) and the depth of memory to which the query will be done .
attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units,output)
#multi layer decoder cell
d_cells = [tf.contrib.rnn.GRUCell(num_units) for _ in range(NLAYERS)]
# apply attention on the top/last layer of the decoder cell and provide it as a wrapper . Output attention true then return attention else if false, then output = output
#Working of attention mechanism :
attention_cell = tf.contrib.seq2seq.AttentionWrapper(d_cells[-1],attention_mechanism,output_attention=False)
d_cells[-1] = attention_cell
decoder_cell = tf.contrib.rnn.MultiRNNCell(d_cells)
## shape of decoder cell --  (NLayer-1) * RNNCell + AttentionWrappedCell
## since the last layer in the cell is of type wrapped, so the last layer in the encoder final state should also be of form wrapped . So modify last layer state to be of type wrapped
decoder_batch_size = tf.placeholder(dtype=tf.int32,shape=[1],name="DECODER_BATCH_SIZE") 
d_state = attention_cell.zero_state(decoder_batch_size,dtype=tf.float32)
decoder_input_state = [x for x in state] # convert tuple to array so that item assignment can be done later
d_state = d_state.clone(cell_state=decoder_input_state[-1])
decoder_input_state[-1] = d_state
decoder_input_state = tuple(decoder_input_state)
basic_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,helper,decoder_input_state,decoder_output_layer)
final_outputs, final_state, final_sequence_lengths= tf.contrib.seq2seq.dynamic_decode(basic_decoder)
## dynamic_decode calls initialise methof on the basic_decoder once, calls step function repetadely for
##sequence_length parameter of training helper times. Initialse method returns (finished, first_inputs, initial_state).
##step function takes inputs as step(time,inputs,state,name=None),performs a decoding step and returns as output (outputs, next_state, next_inputs, finished).
###Attention model is basically a pool of memory where we randomly access a state which will better determine our current state.Alignment  : which part of source we will be next translating 
##previous hidden state will be used 
## score -- score(ht-1 , hs) score each position 
## alignment -- softmax (score) /pd of diff places of memeory
## context vector - sum over alignment * hs
## input to current cell : last hidden state + . context vector + decoder input
##score : a - > h(t-1)Transpose * hs
##        b -> h(t-1)Transpose * W * hs  // interaction between h(t-1) and hs
##        c -> v Transpose * tanh(W * [h(t-1) hs]
##Perform a step of attention-wrapped RNN.

#Step 1: Mix the inputs and previous step's attention output via cell_input_fn - array_ops.concat([inputs, attention],-1): the input and attention both should have same dimensions 
##except the last column along which conactenation should be performed. Here the concatenated input will be of of length inputs.colum length + attention.colum length(internal cell size)
#Step 2: Call the wrapped cell with this input and its previous state.
#Step 3: Score the cell's output with attention_mechanism.
#Step 4: Calculate the alignments by passing the score through the normalizer.
#Step 5: Calculate the context vector as the inner product between the alignments and the attention_mechanism's values (memory).
#Step 6: Calculate the attention output by concatenating the cell output and context through the attention layer (a linear layer with attention_layer_size outputs).

In [8]:
decoder_input_state

(<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 512) dtype=float32>,
 <tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 512) dtype=float32>,
 AttentionWrapperState(cell_state=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 512) dtype=float32>, attention=<tf.Tensor 'AttentionWrapperZeroState/zeros_1:0' shape=(?, 512) dtype=float32>, time=<tf.Tensor 'AttentionWrapperZeroState/zeros:0' shape=() dtype=int32>, alignments=<tf.Tensor 'AttentionWrapperZeroState/zeros_2:0' shape=(?, ?) dtype=float32>, alignment_history=()))

In [9]:
##decoder_logits = tf.contrib.layers.linear(final_outputs.rnn_output,decoder_vocabulary_size)
decoder_logits = final_outputs.rnn_output

In [10]:
decoder_logits

<tf.Tensor 'decoder/transpose:0' shape=(?, ?, 23) dtype=float32>

In [16]:
decoder_prediction = tf.arg_max(tf.nn.softmax(decoder_logits),2)

In [17]:
decoder_targets = tf.placeholder(dtype=tf.int32,shape=[None,None],name = "DecoderTargets")
lr = tf.placeholder(dtype=tf.float32,name='lr')

In [18]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(decoder_targets,depth=decoder_vocabulary_size,dtype=tf.float32),logits=decoder_logits)

In [19]:
loss = tf.reduce_mean(stepwise_cross_entropy)

In [20]:
optimiser = tf.train.AdamOptimizer(lr).minimize(loss)

### Load train and test data

In [21]:
source = open("sources.txt")
train = [[int(word) for word in line.split()] for line in source]
target = open("targets.txt")
target = [[int(word) for word in line.split()] for line in target]

### Process train data and generate batch of data

In [22]:

EOS = 20 ## decoder outputs will end with EOS
PAD = 21 ## PAD will be the extra word after EOS or end of encoder inputs to so that length of seq of all inputs in a batch is same
GO = 22  ## Decoder Inputs will start with a GO
epoch = 20
def modifyBatch(input):
    inputSeqLen = [len(seq) for seq in input]
    maxSourceSeqLen = max(inputSeqLen)
    batchedInput = np.ones(shape=(len(input),maxSourceSeqLen),dtype=np.int32)*PAD
    for i,seq in enumerate(input):
        for j,word in enumerate(seq):
            batchedInput[i][j] = word
    return batchedInput,inputSeqLen
        
def generateBatch(source,target,batch_size,epochs):
    numBatches = int(len(source)/batch_size)
    DecoderInputs = [[GO] + word for word in target ]
    DecoderTargets = [word + [EOS] for word in target]
    for it in range(epochs):
        for i in range(numBatches):
            encoderInputs,encoderInputLength = modifyBatch(source[i*batch_size:(i+1)*batch_size])
            decoderInputs,decoderInputLength = modifyBatch(DecoderInputs[i*batch_size:(i+1)*batch_size])
            decoderTargets,decoderTargetLength = modifyBatch(DecoderTargets[i*batch_size:(i+1)*batch_size])
            yield(encoderInputs,encoderInputLength,decoderInputs,decoderInputLength,decoderTargets,decoderTargetLength,i)

In [23]:
timestamp = str(math.trunc(time.time()))
if not os.path.exists("seq_seq_checkpoints"):
    os.mkdir("seq_seq_checkpoints")
saver = tf.train.Saver(max_to_keep=1000)

In [24]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

### Training

In [25]:
for encoderInputs,encoderInputLength,decoderInputs,decoderInputLength,decoderTargets,decoderTargetLength,batch_num in generateBatch(train,target,BATCH_SIZE,epoch):
    feedDict = {ENCODER_INPUTS:encoderInputs,EncoderSeqLength:encoderInputLength,DecoderSeqLength:decoderInputLength,DECODER_INPUTS:decoderInputs,decoder_targets:decoderTargets,lr:0.001,decoder_batch_size:np.array([BATCH_SIZE])}
    _,l = sess.run([optimiser,loss],feed_dict=feedDict)
    predict_ = sess.run(decoder_prediction,feed_dict=feedDict)
    print("Loss for batch i ", batch_num,l)
    if batch_num // 50 == 0:
        saved_file = saver.save(sess, 'seq_seq_checkpoints/seq_seq_train' + timestamp, global_step=batch_num)
        print("Saved file: " + saved_file)
saved_file = saver.save(sess, 'seq_seq_checkpoints/seq_seq_train' + timestamp, global_step=batch_num)
print("Saved file: " + saved_file) 
        
        

Loss for batch i  0 3.12509
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-0
Loss for batch i  1 2.88251
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-1
Loss for batch i  2 2.55561
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-2
Loss for batch i  3 2.41586
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-3
Loss for batch i  4 2.30658
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-4
Loss for batch i  5 2.20684
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-5
Loss for batch i  6 2.27398
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-6
Loss for batch i  7 2.33265
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-7
Loss for batch i  8 2.12041
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-8
Loss for batch i  9 2.19521
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-9
Loss for batch i  10 2.07583
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-10
Loss for batch i  11 2.07454
Saved file: seq_seq_che

Loss for batch i  180 1.33583
Loss for batch i  181 1.15898
Loss for batch i  182 1.11725
Loss for batch i  183 1.2831
Loss for batch i  184 1.18022
Loss for batch i  185 1.17896
Loss for batch i  186 1.08055
Loss for batch i  187 1.10654
Loss for batch i  188 1.13361
Loss for batch i  189 1.24295
Loss for batch i  190 1.20901
Loss for batch i  191 1.05234
Loss for batch i  192 1.15643
Loss for batch i  193 1.44597
Loss for batch i  194 1.1997
Loss for batch i  195 1.22281
Loss for batch i  196 1.26457
Loss for batch i  197 1.01377
Loss for batch i  198 1.27869
Loss for batch i  199 1.06704
Loss for batch i  0 1.14541
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-0
Loss for batch i  1 1.36935
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-1
Loss for batch i  2 1.26601
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-2
Loss for batch i  3 1.16735
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-3
Loss for batch i  4 1.44876
Saved file: seq_seq_checkpoint

Loss for batch i  155 0.0481912
Loss for batch i  156 0.0491657
Loss for batch i  157 0.0552197
Loss for batch i  158 0.0479785
Loss for batch i  159 0.0390131
Loss for batch i  160 0.040968
Loss for batch i  161 0.0361332
Loss for batch i  162 0.0402215
Loss for batch i  163 0.0360328
Loss for batch i  164 0.0372847
Loss for batch i  165 0.0388913
Loss for batch i  166 0.0291943
Loss for batch i  167 0.0285364
Loss for batch i  168 0.0426318
Loss for batch i  169 0.0500227
Loss for batch i  170 0.0282381
Loss for batch i  171 0.0224281
Loss for batch i  172 0.0516338
Loss for batch i  173 0.0242627
Loss for batch i  174 0.0393281
Loss for batch i  175 0.0292565
Loss for batch i  176 0.0535849
Loss for batch i  177 0.019901
Loss for batch i  178 0.0292108
Loss for batch i  179 0.0312912
Loss for batch i  180 0.034669
Loss for batch i  181 0.0207051
Loss for batch i  182 0.0305843
Loss for batch i  183 0.0246986
Loss for batch i  184 0.0219268
Loss for batch i  185 0.022483
Loss for bat

Loss for batch i  129 0.486648
Loss for batch i  130 0.736463
Loss for batch i  131 1.46314
Loss for batch i  132 1.34401
Loss for batch i  133 1.05794
Loss for batch i  134 1.0669
Loss for batch i  135 1.93496
Loss for batch i  136 1.97284
Loss for batch i  137 1.95819
Loss for batch i  138 1.58857
Loss for batch i  139 1.26403
Loss for batch i  140 0.827934
Loss for batch i  141 1.09314
Loss for batch i  142 1.0379
Loss for batch i  143 1.1397
Loss for batch i  144 0.828622
Loss for batch i  145 0.663601
Loss for batch i  146 0.676688
Loss for batch i  147 0.742539
Loss for batch i  148 0.89335
Loss for batch i  149 0.637776
Loss for batch i  150 0.576373
Loss for batch i  151 1.17395
Loss for batch i  152 0.644996
Loss for batch i  153 0.770395
Loss for batch i  154 0.692845
Loss for batch i  155 0.568577
Loss for batch i  156 0.439648
Loss for batch i  157 0.427085
Loss for batch i  158 0.36065
Loss for batch i  159 0.366325
Loss for batch i  160 0.251583
Loss for batch i  161 0.38

Loss for batch i  106 1.5522
Loss for batch i  107 1.45672
Loss for batch i  108 1.39815
Loss for batch i  109 1.39091
Loss for batch i  110 1.45642
Loss for batch i  111 1.3676
Loss for batch i  112 1.39246
Loss for batch i  113 1.40207
Loss for batch i  114 1.23843
Loss for batch i  115 1.36726
Loss for batch i  116 1.23626
Loss for batch i  117 1.45044
Loss for batch i  118 1.38365
Loss for batch i  119 1.21536
Loss for batch i  120 1.33587
Loss for batch i  121 1.25728
Loss for batch i  122 1.31247
Loss for batch i  123 1.41109
Loss for batch i  124 1.13524
Loss for batch i  125 1.17161
Loss for batch i  126 1.24248
Loss for batch i  127 1.04402
Loss for batch i  128 1.12356
Loss for batch i  129 1.06904
Loss for batch i  130 1.10354
Loss for batch i  131 1.23228
Loss for batch i  132 1.20233
Loss for batch i  133 1.03572
Loss for batch i  134 1.10253
Loss for batch i  135 0.912729
Loss for batch i  136 0.94349
Loss for batch i  137 1.01672
Loss for batch i  138 0.794445
Loss for b

...

Loss for batch i  93 0.737057
Loss for batch i  94 0.851003
Loss for batch i  95 1.1084
Loss for batch i  96 1.17365
Loss for batch i  97 1.04451
Loss for batch i  98 0.97338
Loss for batch i  99 0.801839
Loss for batch i  100 1.03958
Loss for batch i  101 0.918268
Loss for batch i  102 0.811546
Loss for batch i  103 0.67178
Loss for batch i  104 0.829183
Loss for batch i  105 0.76947
Loss for batch i  106 0.483457
Loss for batch i  107 0.49839
Loss for batch i  108 0.532995
Loss for batch i  109 0.555204
Loss for batch i  110 0.491371
Loss for batch i  111 0.569649
Loss for batch i  112 0.52128
Loss for batch i  113 0.437743
Loss for batch i  114 0.313183
Loss for batch i  115 0.506393
Loss for batch i  116 0.452632
Loss for batch i  117 0.538221
Loss for batch i  118 0.332467
Loss for batch i  119 0.335477
Loss for batch i  120 0.378826
Loss for batch i  121 0.392661
Loss for batch i  122 0.250484
Loss for batch i  123 0.332911
Loss for batch i  124 0.226864
Loss for batch i  125 0.3

Loss for batch i  68 1.55363
Loss for batch i  69 1.7783
Loss for batch i  70 1.77989
Loss for batch i  71 2.07483
Loss for batch i  72 1.65637
Loss for batch i  73 1.44641
Loss for batch i  74 1.69202
Loss for batch i  75 1.73021
Loss for batch i  76 1.61985
Loss for batch i  77 1.6432
Loss for batch i  78 1.56309
Loss for batch i  79 1.57575
Loss for batch i  80 1.584
Loss for batch i  81 1.43046
Loss for batch i  82 1.42624
Loss for batch i  83 1.65134
Loss for batch i  84 1.42368
Loss for batch i  85 1.41331
Loss for batch i  86 1.64762
Loss for batch i  87 1.44742
Loss for batch i  88 1.50192
Loss for batch i  89 1.39619
Loss for batch i  90 1.2032
Loss for batch i  91 1.14782
Loss for batch i  92 1.35843
Loss for batch i  93 1.39329
Loss for batch i  94 1.49288
Loss for batch i  95 1.37566
Loss for batch i  96 1.43133
Loss for batch i  97 1.22351
Loss for batch i  98 1.38759
Loss for batch i  99 1.25332
Loss for batch i  100 1.39829
Loss for batch i  101 1.36675
Loss for batch i 

Loss for batch i  48 0.351595
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-48
Loss for batch i  49 0.182083
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-49
Loss for batch i  50 0.328289
Loss for batch i  51 0.36957
Loss for batch i  52 0.319704
Loss for batch i  53 0.401325
Loss for batch i  54 0.180018
Loss for batch i  55 0.436489
Loss for batch i  56 0.233594
Loss for batch i  57 0.357179
Loss for batch i  58 0.526797
Loss for batch i  59 0.374151
Loss for batch i  60 0.310472
Loss for batch i  61 0.385631
Loss for batch i  62 0.560384
Loss for batch i  63 0.373312
Loss for batch i  64 0.495084
Loss for batch i  65 0.752275
Loss for batch i  66 0.747646
Loss for batch i  67 1.55598
Loss for batch i  68 2.0034
Loss for batch i  69 2.24795
Loss for batch i  70 2.34644
Loss for batch i  71 2.649
Loss for batch i  72 2.02425
Loss for batch i  73 1.72397
Loss for batch i  74 2.02874
Loss for batch i  75 2.12186
Loss for batch i  76 1.95524
Loss for batch i  77 2.003

Loss for batch i  40 0.348569
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-40
Loss for batch i  41 0.421856
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-41
Loss for batch i  42 0.346894
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-42
Loss for batch i  43 0.221108
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-43
Loss for batch i  44 0.292881
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-44
Loss for batch i  45 0.268709
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-45
Loss for batch i  46 0.235064
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-46
Loss for batch i  47 0.254681
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-47
Loss for batch i  48 0.294381
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-48
Loss for batch i  49 0.272855
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-49
Loss for batch i  50 0.367021
Loss for batch i  51 0.321028
Loss for batch i  52 0.191154
Loss for batch i  53

Saved file: seq_seq_checkpoints/seq_seq_train1503085280-31
Loss for batch i  32 0.206807
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-32
Loss for batch i  33 0.142259
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-33
Loss for batch i  34 0.236897
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-34
Loss for batch i  35 0.229535
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-35
Loss for batch i  36 0.333825
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-36
Loss for batch i  37 0.398877
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-37
Loss for batch i  38 0.59881
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-38
Loss for batch i  39 0.387574
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-39
Loss for batch i  40 0.24723
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-40
Loss for batch i  41 0.593907
Saved file: seq_seq_checkpoints/seq_seq_train1503085280-41
Loss for batch i  42 0.492516
Saved file: seq_seq_che

In [26]:
decoder_input_state

(<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 512) dtype=float32>,
 <tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 512) dtype=float32>,
 AttentionWrapperState(cell_state=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 512) dtype=float32>, attention=<tf.Tensor 'AttentionWrapperZeroState/zeros_1:0' shape=(?, 512) dtype=float32>, time=<tf.Tensor 'AttentionWrapperZeroState/zeros:0' shape=() dtype=int32>, alignments=<tf.Tensor 'AttentionWrapperZeroState/zeros_2:0' shape=(?, ?) dtype=float32>, alignment_history=()))

### INFERENCE

In [27]:

decoderStartTokens = tf.placeholder(tf.int32,shape=[None], name="start_token")
decoderEndToken = EOS
## since decoder during inference has no input, so we use GreedyEmbeddingHelper which takes as input only the embeddingmatrix.
## during training, input to decoder is seq of words started with GO, output target output shifted by 1 and ending with EOS.
##So we specify those charcaters also as input for the GreedyEmbedding helper;
## Uses the argmax of the output (treated as logits) and passes the result through an embedding layer to get the next input.
decoderHelper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,decoderStartTokens,decoderEndToken)
inference_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,decoderHelper,decoder_input_state,decoder_output_layer)
final_inference_outputs,final_inference_state,final_inference_sequence_lengths= tf.contrib.seq2seq.dynamic_decode(inference_decoder)


In [28]:
test_input = [[0,2,4,6,8,10],[1,3,5,7,9,11,13,15,17,2]]
test_input,input_length = modifyBatch(test_input)
decoder_start_token = np.array([GO,GO])
feed_Dict = {ENCODER_INPUTS: test_input,EncoderSeqLength:np.array(input_length),decoderStartTokens:decoder_start_token,decoder_batch_size:np.array([2])}





In [29]:
test_input

array([[ 0,  2,  4,  6,  8, 10, 21, 21, 21, 21],
       [ 1,  3,  5,  7,  9, 11, 13, 15, 17,  2]], dtype=int32)

In [30]:
final_inference_outputs

BasicDecoderOutput(rnn_output=<tf.Tensor 'decoder_1/transpose:0' shape=(?, ?, 23) dtype=float32>, sample_id=<tf.Tensor 'decoder_1/transpose_1:0' shape=(?, ?) dtype=int32>)

In [31]:
#new_saver = tf.train.import_meta_graph('seq_seq_checkpoints1/seq_seq_train1500951341-49.meta')
#new_saver.restore(sess, "seq_seq_checkpoints1/seq_seq_train1500951341-49")
predict_test,seqLength = sess.run([final_inference_outputs,final_inference_sequence_lengths],feed_dict = feed_Dict)



In [32]:
predict_test.sample_id

array([[10,  8,  6,  4,  2,  0, 20, 20, 20, 20, 20],
       [ 2, 17, 15, 13, 11,  9,  7,  5,  3,  1, 20]], dtype=int32)

In [33]:
predict_test[0].shape

(2, 11, 23)

In [ ]:
###Attention


In [ ]:
predict_test

In [ ]:
seqLength